In [177]:
import numpy as np

from pycobyla.gcobyla import GCobyla

# Gaussian simplex example

In [178]:
def gaussian(x, mu=None, sig=None, A=1):
    n = len(x)
    mu =  np.zeros(n) if mu is None else mu
    sig = np.ones(n) if sig is None else sig

    zz = ((((x - mu) / sig) ** 2) / 2).sum()
    return A * np.exp(-zz)


def neg_gaussian(x, mu=None, sig=None, A=1):
    return -gaussian(x, mu=mu, sig=sig, A=A)


def gaussian_optimizer():
    F = neg_gaussian
    c1 = lambda x: 1 - x[0]
    c2 = lambda x: 1 + x[0]
    c3 = lambda x: 1 - x[1]
    c4 = lambda x: 1 + x[1]

    C = (c1, c2, c3, c4)
    x = np.ones(2)    
    mu = np.array((0, 0))
    
    opt = GCobyla(x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt


In [179]:
%matplotlib widget
#%matplotlib notebook
import itertools

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


def simplex_3d_plot(opt):
    plt.close('all')
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    points = np.array((opt.optimal_vertex, *(opt.sim + opt.optimal_vertex), opt.optimal_vertex))
    points = np.c_[points, np.zeros(points.shape[0])]
    ax.plot(points[0,0], points[0,1], points[0, 2], 'go')
    ax.plot(points[1:-1,0], points[1:-1,1], points[1:-1, 2], 'bo')
    ax.plot(points[...,0], points[...,1], points[..., 2], 'r-', lw=2)
    
    # Make data.
    max_p = points.max(axis=0)[:-1].max()
    min_p = points.min(axis=0)[:-1].min()

    X = np.linspace(min_p, max_p, 50)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)

    # Plot the surface.
    surf = ax.plot_surface(xv, yv, zv, cmap=cm.plasma, linewidth=0, antialiased=False)

    # Customize the z axis.
    ax.set_zlim(-1., 0.1)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    plt.show()

In [180]:
opt = gaussian_optimizer()
steps_it =(step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

simplex_3d_plot(opt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [181]:
next(steps_it)
simplex_3d_plot(opt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [182]:
def simplex_2d_plot(opt, target):
    plt.close('all')
    fig, ax = plt.subplots()
    
    points = np.array((opt.optimal_vertex, *(opt.sim + opt.optimal_vertex), opt.optimal_vertex))
    ax.plot(*opt.optimal_vertex, color='orange', marker='o')
    ax.plot(points[1:-1,0], points[1:-1,1], '.', color='blue', marker='o')
    ax.plot(points[...,0], points[...,1], '-.', color='red', lw=2)
    ax.plot(target[0], target[1], color='limegreen', marker='*')
    
    # Make data
    all_points = np.array((*points, target))
    max_p = all_points.max(axis=0).max()
    min_p = all_points.min(axis=0).min()
    diff_p = max_p - min_p

    X = np.linspace(max_p + (0.1 * diff_p), min_p - (0.1 * diff_p), 50)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(-opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)
    
    cs = ax.contourf(xv, yv, zv, cmap=cm.PuBu_r)
    cbar = fig.colorbar(cs)
    
    plt.show()


In [183]:
opt = gaussian_optimizer()
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

simplex_2d_plot(opt, target=(0,0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [184]:
next(steps_it)
simplex_2d_plot(opt, target=(0,0))
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 3
x: [1. 1.]
optimal_vertex: [1. 1.]
datmat: 
[[-0.5         2.5         0.          2.         -0.19691168  0.5       ]
 [ 0.          2.         -0.5         2.5        -0.19691168  0.5       ]
 [ 0.          2.          0.          2.         -0.36787944  0.        ]]
a: 
None
sim: 
[[0.5 0. ]
 [0.  0.5]]
simi: 
[[2. 0.]
 [0. 2.]]
optimal vertex: 
[1. 1.]
parmu: 0
rho: 0.5


In [185]:
for _ in steps_it: pass
simplex_2d_plot(opt, target=(0,0))
opt.data
print(opt.sim @ opt.optimal_vertex)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 88
x: [1.40230486e-10 8.95344630e-11]
optimal_vertex: [1.40230486e-10 8.95344630e-11]
datmat: 
[[ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]]
a: 
[[-1.00000002e+00 -1.16238286e-08]
 [ 9.99999908e-01  5.78209700e-09]
 [-7.92720677e-08 -9.99999987e-01]
 [-2.66223273e-08  1.00000001e+00]
 [-0.00000000e+00 -0.00000000e+00]]
sim: 
[[-4.88351701e-13  1.07296861e-13]
 [-3.90343708e-13 -1.77661314e-12]]
simi: 
[[-1.95340680e+12 -1.17974147e+11]
 [ 4.29187445e+11 -5.36948373e+11]]
optimal vertex: 
[1.40230486e-10 8.95344630e-11]
parmu: 0
rho: 1e-12
[-5.88750293e-23 -2.13806191e-22]


# Gaussian track examples 

In [266]:
%matplotlib widget

import matplotlib.pyplot as plt

def plot_track(opt, target, n_points=5, plot_simplex=True, plot_trure=True):
    def plot_track(ax, n_points):
        track = opt.track[-n_points:]
        ax.plot(track[:, 0], track[:, 1], linestyle='-', color='red', marker='o')
        link_best = np.array((track[-1], target))
        ax.plot(link_best[:, 0], link_best[:, 1], linestyle=':', color='red')
        ax.plot(target[0], target[1], color='limegreen', marker='*')
        
        if plot_simplex:
            points = np.array((opt.optimal_vertex, *(opt.sim + opt.optimal_vertex), opt.optimal_vertex))
            ax.plot(*opt.optimal_vertex, color='orange', marker='o')
            ax.plot(points[1:-1,0], points[1:-1,1], '.', color='blue', marker='o')
            ax.plot(points[...,0], points[...,1], '-.', color='gray', lw=2)
            
        if plot_trure:
            trust_region = plt.Circle(opt.optimal_vertex, opt.rho, color='khaki', fill=True, alpha=0.5)
            ax.add_patch(trust_region)
    ###
    
    plt.close('all')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    plot_track(ax1, n_points=n_points)
    plot_track(ax2, n_points=0)

    plt.show()
    return fig, (ax1, ax2)

In [267]:
opt = gaussian_optimizer()
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

In [278]:
next(steps_it)
plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=False)
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 8
x: [ 0.09678972 -0.49339449]
optimal_vertex: [-0.08232556 -0.02657791]
datmat: 
[[ 1.50149105  0.49850895  0.75399711  1.24600289 -0.85555462  0.        ]
 [ 0.76075325  1.23924675  0.64370554  1.35629446 -0.9120209   0.        ]
 [ 1.08232556  0.91767444  1.02657791  0.97342209 -0.99626505  0.        ]]
a: 
[[-1.00000000e+00  5.42101086e-20]
 [ 1.00000000e+00 -5.42101086e-20]
 [-8.13151629e-20 -1.00000000e+00]
 [ 8.13151629e-20  1.00000000e+00]
 [ 1.24569336e-01 -3.24657013e-01]]
sim: 
[[-0.41916549  0.2725808 ]
 [ 0.32157231  0.38287237]]
simi: 
[[-1.54296095  1.09849016]
 [ 1.29592406  1.68922084]]
optimal vertex: 
[-0.08232556 -0.02657791]
parmu: 0
rho: 0.25


In [189]:
for _ in steps_it: pass
plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True)
opt.data
print(opt.sim / opt.sim.min())
print(opt.sim @ opt.sim.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 88
x: [1.40230486e-10 8.95344630e-11]
optimal_vertex: [1.40230486e-10 8.95344630e-11]
datmat: 
[[ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]
 [ 1.  1.  1.  1. -1.  0.]]
a: 
[[-1.00000002e+00 -1.16238286e-08]
 [ 9.99999908e-01  5.78209700e-09]
 [-7.92720677e-08 -9.99999987e-01]
 [-2.66223273e-08  1.00000001e+00]
 [-0.00000000e+00 -0.00000000e+00]]
sim: 
[[-4.88351701e-13  1.07296861e-13]
 [-3.90343708e-13 -1.77661314e-12]]
simi: 
[[-1.95340680e+12 -1.17974147e+11]
 [ 4.29187445e+11 -5.36948373e+11]]
optimal vertex: 
[1.40230486e-10 8.95344630e-11]
parmu: 0
rho: 1e-12
[[ 0.27487791 -0.06039405]
 [ 0.21971227  1.        ]]
[[2.50000000e-25 0.00000000e+00]
 [0.00000000e+00 3.30872245e-24]]


# Pyramid track example

In [190]:
%matplotlib widget
import itertools
import functools

import numpy as np
import matplotlib.pyplot as plt

import tests.test_custom as tc


def pyramid_optimizer(start_x=None, width=2):
    F = functools.partial(tc.pyramid, center=np.zeros(2), width=width, height=-1)
    C = ()
    start_x = np.random.uniform(low=-width/2, high=width/2, size=2) if start_x is None else start_x
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [191]:
start_x = np.array((-0.57714299498416465894479, -0.89532481179992373654386))
opt = pyramid_optimizer(start_x=start_x)
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

In [192]:
next(steps_it)
plot_track(opt, target=np.zeros(2), n_points=2, plot_simplex=True, plot_trure=True)
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 3
x: [-0.57714299 -0.39532481]
optimal_vertex: [-0.57714299 -0.39532481]
datmat: 
[[-0.10467519  0.        ]
 [-0.10467519  0.        ]
 [-0.42285701  0.        ]]
a: 
None
sim: 
[[ 0.5 -0.5]
 [ 0.  -0.5]]
simi: 
[[ 2. -2.]
 [ 0. -2.]]
optimal vertex: 
[-0.57714299 -0.39532481]
parmu: 0
rho: 0.5


In [193]:
for _ in steps_it: pass
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True, plot_trure=True)
ax2.plot(np.array((-1, 1)), np.array((1, -1)), linestyle='-.')
ax2.plot(np.array((-1, 1)), np.array((-1, 1)), linestyle='-.')

margin = 1e-1
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))

opt.data
print(opt.sim / opt.sim.min())
print(opt.sim @ opt.sim.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 437
x: [-0.19197598 -0.19197598]
optimal_vertex: [-0.19197598 -0.19197598]
datmat: 
[[-0.80802402  0.        ]
 [-0.80802402  0.        ]
 [-0.80802402  0.        ]]
a: 
[[0.03450131 0.86683888]]
sim: 
[[-9.68035495e-13 -1.25081297e-12]
 [ 3.97698083e-14  9.99208868e-13]]
simi: 
[[-1.08902624e+12 -1.36324666e+12]
 [ 4.33446563e+10  1.05505074e+12]]
optimal vertex: 
[-0.19197598 -0.19197598]
parmu: 0
rho: 1e-12
[[ 0.77392505  1.        ]
 [-0.03179517 -0.79884754]]
[[ 2.50162581e-24 -1.28832200e-24]
 [-1.28832200e-24  1.00000000e-24]]


# 4 faces pyramid track example

In [280]:
%matplotlib widget
import itertools
import functools

import numpy as np
import matplotlib.pyplot as plt

import tests.test_custom as tc


def pyramid_faces_optimizer(start_x, radius=2, faces=4):
    F = functools.partial(tc.pyramid_faces, center=np.zeros(2), radius=radius, height=-1, faces=faces)
    c1 = lambda x: 1 - sum(x ** 2)
    C = (c1,)
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [281]:
#start_x = np.array((0.29571617183264287120892, 0.06643579296841295445120))
start_x = np.array((0.60522848317453536992616, -0.92243950619371162247262))
opt = pyramid_faces_optimizer(start_x=start_x, faces=4)
steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})

In [408]:
next(steps_it)
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True, plot_trure=True)
ax2.plot(np.array((-1, 1)), np.array((0, 0)), linestyle='-.')
ax2.plot(np.array((0, 0)), np.array((-1, 1)), linestyle='-.')

margin = 1e-2
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))
opt.data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 77
x: [8.38589771e-02 3.93962141e-06]
optimal_vertex: [ 8.38829945e-02 -1.48882958e-05]
datmat: 
[[ 0.99296767 -0.95806854  0.        ]
 [ 0.99295933 -0.95804487  0.        ]
 [ 0.99296364 -0.95805106  0.        ]]
a: 
[[-1.67852023e-01  1.07639041e-04]
 [-4.82821659e-01  3.78498629e-01]]
sim: 
[[-2.40173293e-05  1.88279172e-05]
 [ 2.57078440e-05  1.64447356e-05]]
simi: 
[[-18708.8041824   21420.09605582]
 [ 29247.23336508  27323.9729616 ]]
optimal vertex: 
[ 8.38829945e-02 -1.48882958e-05]
parmu: 0
rho: 3.0517578125e-05


In [409]:
for _ in steps_it: pass
fig, (ax1, ax2) = plot_track(opt, target=np.zeros(2), n_points=5, plot_simplex=True, plot_trure=True)
ax2.plot(np.array((-1, 1)), np.array((0, 0)), linestyle='-.')
ax2.plot(np.array((0, 0)), np.array((-1, 1)), linestyle='-.')

margin = 1e-1
xmin = min((*opt.track[:, 0], 0))
xmax = max((*opt.track[:, 0], 0))
ymin = min((*opt.track[:, 1], 0))
ymax = max((*opt.track[:, 1], 0))
ax2.set(xlim=(xmin - margin, xmax + margin), ylim=(ymin - margin, ymax + margin))

opt.data
print(opt.sim / opt.sim.min())
print(opt.sim @ opt.sim.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nfvals: 371
x: [8.34524795e-02 7.67764511e-13]
optimal_vertex: [8.34524795e-02 1.02106460e-13]
datmat: 
[[ 0.99303568 -0.95827376  0.        ]
 [ 0.99303568 -0.95827376  0.        ]
 [ 0.99303568 -0.95827376  0.        ]]
a: 
[[-1.66904975e-01 -1.01481399e-09]
 [-4.52220685e-01  4.03378970e-01]]
sim: 
[[-7.46256899e-13  6.65658050e-13]
 [ 1.60349663e-12 -2.63840311e-13]]
simi: 
[[3.03094787e+11 7.64695447e+11]
 [1.84206677e+12 8.57285889e+11]]
optimal vertex: 
[8.34524795e-02 1.02106460e-13]
parmu: 0
rho: 1e-12
[[ 1.         -0.89199584]
 [-2.14871934  0.35355159]]
[[ 1.00000000e-24 -1.37224785e-24]
 [-1.37224785e-24  2.64081317e-24]]


In [214]:
print(f'{opt.F(opt.optimal_vertex):.60}')
print(f'{opt.F(opt.optimal_vertex + opt.sim[0]):.60}')
print(f'{opt.F(opt.optimal_vertex + opt.sim[1]):.60}')

-0.95827376022649113362916750702424906194210052490234375
-0.9582737602265314347249614002066664397716522216796875
-0.95827376022565957658372326477547176182270050048828125
